# bs4 데이터 수집, mongo DB 연동

### 데이터 수집을 위한 분석
#### 데이터 수집 할 주제
- 영화 배우 정보

#### 수집 데이터
- 정보 사이트
    - 메인 페이지
        - http://www.cine21.com/rank/person
    - 해당 배우 세부 정보
        - http://www.cine21.com/db/person/info/?person_id=71827
- 항목 
    - 메인 페이지 : 이름(2)편 -> 이름, 흥행 지수
    - 세부 페이지 : 배우 기본 정보(항목, 값), 참여 작품
#### 수집, 데이터 저장 DB
- mongoDB : 정형화 되지 않은 데이터
- pymongo : python 연동

#### 수집 도구 : library
- requests
- bs4

### 데이터 수집을 위한 절차 정리

#### request url
- "http://www.cine21.com/rank/person"
- "http://www.cine21.com/db/person/info/?person_id=71827"


#### 작업 순서
- 필요한 라이브러리 import
- url request
- response url 객체 생성(parsing / html.parsor)
- 원하는 데이터 pattern 파악
- select로 원하는 데이터 추출
- 반복문으로 원하는 데이터 변형
    - 항목 : 이름, 흥행지수, 이름에 링크된 url
    - 이름에 링크된 url 활용하여 request 날리기
    - 응답 받아서 세부 정보 추출
        - 배우 기본 정보, 참여 작품

In [226]:
import requests as req
import re
from bs4 import BeautifulSoup as bs
import pymongo

In [227]:
url = "http://www.cine21.com/rank/person"

In [228]:
res = req.get(url)

In [229]:
soup = bs(res.text, "html.parser")

In [230]:
# actor_li = soup.select(".people_list people_li")
actor_li = soup.select("#rank_holder")

In [231]:
for actor in actor_li:
    print(actor)

<div id="rank_holder"></div>


section: actor
period_start: 2023-04
gender: all
page: 1

### 데이터 요청 수정
- actor에 대한 정보는 비동기 방식으로 처리
- method : post
- url : http://www.cine21.com/rank/person/content
- 비동기 정보
    - section: actor
    - period_start: 2023-04 : 시작일
    - gender: all
    - page: 1 : paging

In [298]:
cine21_url = "http://www.cine21.com/rank/person/content" # post netword url
# print(cine21_url)

http://www.cine21.com/rank/person/content


In [238]:
start_month = "2023-04"
url_de = "http://www.cine21.com/"

In [239]:
# post network 에서 가져온 옵션값 저장
cdts = {}
cdts["section"] = "actor"
cdts["period_start"] = start_month
cdts["gender"] = "all"
cdts["page"] = 1

In [240]:
res = req.post(cine21_url, data=cdts)

In [241]:
soup = bs(res.text, "html.parser")

In [248]:
actor_li = soup.select(".people_list .people_li")
# print(actor_li)
for actor in actor_li:
    # print(actor)
    # 배우 이름
    name = actor.select_one(".name").get_text(strip=True).split("(")[0]
    print(name)
    
    # 배우 이름 a 태그 href 속성값 url
    actor_url = actor.select_one(".name a").attrs["href"]
    actor_url = url_de + actor_url
    print(actor_url)
    
    # 흥행 지수 : int 형으로 바꿈
    indices = actor.select_one(".tit+strong").get_text(strip=True).replace(",", "")
    print(indices)
    print("-"*30)
    
    # test = actor.select_one("a img.thumb").attrs["src"]
    # print(test)    
    # test = actor.select_one("a img.thumb")["src"]
    # print(test)



안재홍
http://www.cine21.com//db/person/info/?person_id=4883
79451
------------------------------
이신영
http://www.cine21.com//db/person/info/?person_id=113283
68100
------------------------------
박서준
http://www.cine21.com//db/person/info/?person_id=84745
66234
------------------------------
아이유
http://www.cine21.com//db/person/info/?person_id=71827
59612
------------------------------
정진운
http://www.cine21.com//db/person/info/?person_id=71801
57437
------------------------------
김종수
http://www.cine21.com//db/person/info/?person_id=65628
52989
------------------------------
김보라
http://www.cine21.com//db/person/info/?person_id=73838
49772
------------------------------


## 세부 정보 수집
- 기본 정보 : key, value 같이 수집 - dict()
- 참여 작품 : value만 수집 - list()

In [275]:
actor_link = "http://www.cine21.com//db/person/info/?person_id=71827"

In [276]:
res_sub = req.get(actor_link)
print(res_sub)
soup_sub = bs(res_sub.text, "html.parser")
# print(soup)

<Response [200]>


<!-- 기본정보 -->
<div class="default_info_area">
<h2><span>기본정보</span></h2>
<ul class="default_info">
<li><span class="tit">다른 이름</span>이지은</li>
<li><span class="tit">직업</span>가수</li>
<li><span class="tit">생년월일</span>1993-05-16</li>
<li><span class="tit">성별</span>여</li>
<li><span class="tit">홈페이지</span>
<a href="https://twitter.com/lily199iu" target="_blank">https://twitter.com/lily199iu</a><br/>
<a href="https://www.instagram.com/dlwlrma/" target="_blank">https://www.instagram.com/dlwlrma/</a><br/>
</li>
<li><span class="tit">학교</span>동덕여자고등학교</li>
</ul>
</div>
<!-- //기본정보 -->

In [264]:
info_area = soup_sub.select(".default_info_area .default_info li")

In [261]:
actor_info_dict = {}
homepList = []
for info in info_area:
    homepage = info.select("li a")
    # 예외 상황
    # value의 2가지 상황
    # li span, li span+a
    # 직업
    # 생년월일
    # 성별
    
    if info.select_one("li span+a") == None:
        # 기본 정보
        # 다른 이름
        key = info.select_one("li span.tit").get_text(strip=True)
        # print(key)
        # value = info.get_text(strip=True)
        # print(value)
        value = info.strings # list 로 넘겨줌
        value = list(value)[1]
        # print(value)
        actor_info_dict[key] = value
    

    # 홈페이지
    for home in homepage:
        homepage = home.attrs["href"]
        homepList.append(homepage)
    actor_info_dict["홈페이지"] = homepList
    
    # print(info.select_one("li span+a") == None)
print(actor_info_dict)
    

{'다른 이름': '이지은', '홈페이지': ['https://twitter.com/lily199iu', 'https://www.instagram.com/dlwlrma/'], '직업': '가수', '생년월일': '1993-05-16', '성별': '여', '학교': '동덕여자고등학교'}


In [270]:
default_infos = soup_sub.select("ul.default_info li")
# print(default_infos)

actor_info_dict = {}
for d_info in default_infos:
    # print(d_info)
    # 직업 : 배우, 생년월일 : 1986-03-31 , .....
    key = d_info.select_one("span.tit").get_text(strip=True)
    # print(key)
    
    # value의 두가지 case : li span, li span + a
    """
    value = d_info.select_one("li span + a") => None, None 아닌 경우
    if value None 경우:
        value = list(d_info.strings)[-1]
    else:
        # a 태그의 href 속성의 값 추출
        value = value.attrs[href]
    """ 
    value = d_info.select_one("li span + a")
    if value is None:
        # d_info.strings : 목록형으로 리턴함, list()로 형변환
        value = list(d_info.strings)[-1]
        # print(value)
    else:
        value = value.attrs["href"]

        # print(value)

    # d_info.strings : 목록형으로 리턴함, list()로 형변환
    # value = list(d_info.strings)[-1]
    # print(value)
    actor_info_dict[key] = value
    # print(actor_info_dict)
# print("-"*30)
print(actor_info_dict)
    

{'다른 이름': '이지은', '직업': '가수', '생년월일': '1993-05-16', '성별': '여', '홈페이지': 'https://twitter.com/lily199iu', '학교': '동덕여자고등학교'}


In [267]:
part_work = soup_sub.select("ul.part_works li")
# print(part_work)
# actor_work_dict = {}
workList = []
for work in part_work:
    # print(work)
    # 영화 링크
    movie_link = work.select_one("li a").attrs["href"]
    movie_link = url_de + movie_link
    # print(movie_link)
    
    # 영화 제목
    work_name = work.select_one("span.tit").get_text(strip=True)
    
    # 개봉년도
    work_year = work.select_one("span.year").get_text(strip=True)
    
    workList.append([movie_link, work_name, work_year])
print(workList)
    

[['http://www.cine21.com//movie/info/?movie_id=55390', '드림', '(2023)'], ['http://www.cine21.com//movie/info/?movie_id=56800', '브로커', '(2022)'], ['http://www.cine21.com//db/tv/info/?tv_id=22398', '[스페셜] 아이유 인터뷰', '(2020)']]


### 새로운 배우 데이터 수집

In [273]:
actor_info_dict = {}
homepList = []
for info in info_area:
    homepage = info.select("li a")
    # 기본 정보
    if info.select_one("li span+a") == None:
        key = info.select_one("li span.tit").get_text(strip=True)
        value = info.strings # list 로 넘겨줌
        value = list(value)[1]
        actor_info_dict[key] = value

    # 홈페이지
    for home in homepage:
        homepage = home.attrs["href"]
        homepList.append(homepage)
    actor_info_dict["홈페이지"] = homepList
    
print(actor_info_dict)

part_work = soup_sub.select("ul.part_works li")
workList = []
for work in part_work:
    # 영화 링크
    movie_link = work.select_one("li a").attrs["href"]
    movie_link = url_de + movie_link
    
    # 영화 제목
    work_name = work.select_one("span.tit").get_text(strip=True)
    
    # 개봉년도
    work_year = work.select_one("span.year").get_text(strip=True)
    
    workList.append([movie_link, work_name, work_year])
print(workList)
    

{'다른 이름': '이지은', '홈페이지': ['https://twitter.com/lily199iu', 'https://www.instagram.com/dlwlrma/'], '직업': '가수', '생년월일': '1993-05-16', '성별': '여', '학교': '동덕여자고등학교'}
[['http://www.cine21.com//movie/info/?movie_id=55390', '드림', '(2023)'], ['http://www.cine21.com//movie/info/?movie_id=56800', '브로커', '(2022)'], ['http://www.cine21.com//db/tv/info/?tv_id=22398', '[스페셜] 아이유 인터뷰', '(2020)']]


### 데이터 수집 코드 완료 후 해야할 것
- main page, 상세 정보 실행 통합 코드 생성
- mongoDB 저장하기 위한 데이터 자료 구조 생성
- mongoDB 저장 코드 생성
    - mongoDB 생성 : movies
    - collection 생성 : actors_info
    - Document 입력 : dict()

In [293]:
conn = pymongo.MongoClient('mongodb://localhost:27017')

# movies DB생성
movies = conn.movies

# actors_info 컬렉션 생성
actors_info = movies.actors_info

In [283]:
cine21_url = "http://www.cine21.com/rank/person/content" # post netword url

start_month = "2023-04"
url_de = "http://www.cine21.com/"

# post network 에서 가져온 옵션값 저장
cdts = {}
cdts["section"] = "actor"
cdts["period_start"] = start_month
cdts["gender"] = "all"
cdts["page"] = 1

res = req.post(cine21_url, data=cdts)

soup = bs(res.text, "html.parser")

actor_li = soup.select(".people_list .people_li")
for actor in actor_li:
    actor_info_doc={}
    # print(actor)
    # 배우 이름
    name = actor.select_one(".name").get_text(strip=True).split("(")[0]
    print(name)
    
    # 배우 이름 a 태그 href 속성값 url
    actor_url = actor.select_one(".name a").attrs["href"]
    actor_url = url_de + actor_url
    print(actor_url)
    
    # 흥행 지수 : int 형으로 바꿈
    indices = actor.select_one(".tit+strong").get_text(strip=True).replace(",", "")
    print(indices)
    
    print("-"*30)    
    
    # 배우의 상세 페이지
    actor_link = actor_url
    res_sub = req.get(actor_link)
    soup_sub = bs(res_sub.text, "html.parser")
    
    info_area = soup_sub.select(".default_info_area .default_info li")
    
    actor_info_dict = {}
    homepList = []
    for info in info_area:
        homepage = info.select("li a")
        # 기본 정보
        if info.select_one("li span+a") == None:
            key = info.select_one("li span.tit").get_text(strip=True)
            value = info.strings # list 로 넘겨줌
            value = list(value)[1]
            actor_info_dict[key] = value

        # 홈페이지
        for home in homepage:
            homepage = home.attrs["href"]
            homepList.append(homepage)
        actor_info_dict["홈페이지"] = homepList



    part_work = soup_sub.select("ul.part_works li")
    workList = []
    for work in part_work:
        # 영화 링크
        movie_link = work.select_one("li a").attrs["href"]
        movie_link = url_de + movie_link

        # 영화 제목
        work_name = work.select_one("span.tit").get_text(strip=True)

        # 개봉년도
        work_year = work.select_one("span.year").get_text(strip=True)

        workList.append([movie_link, work_name, work_year])
print(actor_info_dict)
print(workList)
    
    

안재홍
http://www.cine21.com//db/person/info/?person_id=4883
79451
------------------------------
이신영
http://www.cine21.com//db/person/info/?person_id=113283
68100
------------------------------
박서준
http://www.cine21.com//db/person/info/?person_id=84745
66234
------------------------------
아이유
http://www.cine21.com//db/person/info/?person_id=71827
59612
------------------------------
정진운
http://www.cine21.com//db/person/info/?person_id=71801
57437
------------------------------
김종수
http://www.cine21.com//db/person/info/?person_id=65628
52989
------------------------------
김보라
http://www.cine21.com//db/person/info/?person_id=73838
49772
------------------------------
{'직업': '배우', '홈페이지': ['https://www.instagram.com/aahnjaehong/', 'https://www.instagram.com/leesin_y/', 'https://www.instagram.com/bn_sj2013/', 'https://twitter.com/bn_sj2013', 'https://twitter.com/lily199iu', 'https://www.instagram.com/dlwlrma/', 'https://twitter.com/2amjinwoon', 'https://www.facebook.com/jinwoon.jeong', 'http

## mongoDB 저장하기 위한 데이터 변환
- mongoDB : json -> dict() 자료 구조 만들기
- Document로 저장하기 위해 key 정하기
- 메인 페이지
    - actor : 배우명
    - actor_url : 상세 페이지 url
    - indices : 흥행 지수
    - start_period : 데이터 누적 기간
- 상세 페이지
    - info : 배우 기본 정보
    - movie_list : 참여 작품

In [291]:
cine21_url = "http://www.cine21.com/rank/person/content" # post netword url

start_month = "2023-04"
url_de = "http://www.cine21.com/"

# post network 에서 가져온 옵션값 저장
cdts = {}
cdts["section"] = "actor"
cdts["period_start"] = start_month
cdts["gender"] = "all"
cdts["page"] = 1

res = req.post(cine21_url, data=cdts)

soup = bs(res.text, "html.parser")
docsList = [] # 전체 리스트
actor_li = soup.select(".people_list .people_li")
for actor in actor_li:
    actor_info_doc={}
    # print(actor)
    # 배우 이름
    name = actor.select_one(".name").get_text(strip=True).split("(")[0]
    # print(name)
    
    # 배우 이름 a 태그 href 속성값 url
    actor_url = actor.select_one(".name a").attrs["href"]
    actor_url = url_de + actor_url
    # print(actor_url)
    
    # 흥행 지수 : int 형으로 바꿈
    indices = actor.select_one(".tit+strong").get_text(strip=True).replace(",", "")
    # print(indices)
    
   
    
    # 배우의 상세 페이지
    actor_link = actor_url
    res_sub = req.get(actor_link)
    soup_sub = bs(res_sub.text, "html.parser")
    
    info_area = soup_sub.select(".default_info_area .default_info li")
    
    actor_info_dict = {}
    homepList = []
    for info in info_area:
        homepage = info.select("li a")
        # 기본 정보
        if info.select_one("li span+a") == None:
            key = info.select_one("li span.tit").get_text(strip=True)
            value = info.strings # list 로 넘겨줌
            value = list(value)[1]
            actor_info_dict[key] = value

        # 홈페이지
        for home in homepage:
            homepage = home.attrs["href"]
            homepList.append(homepage)
        actor_info_dict["홈페이지"] = homepList



    part_work = soup_sub.select("ul.part_works li")
    workList = [] # 참여 작품
    for work in part_work:
        # 영화 링크
        movie_link = work.select_one("li a").attrs["href"]
        movie_link = url_de + movie_link

        # 영화 제목
        work_name = work.select_one("span.tit").get_text(strip=True)

        # 개봉년도
        work_year = work.select_one("span.year").get_text(strip=True)

        workList.append([movie_link, work_name, work_year])
# print(actor_info_dict)
# print(workList)
    
    

    # 한 배우에 대한 data부터 만들어보기 : dict()

    actor_info_doc["actor"] = name
    actor_info_doc["actor_url"] = actor_url
    actor_info_doc["indices"] = indices
    actor_info_doc["start_period"] = start_month
    actor_info_doc["info"] = actor_info_dict
    actor_info_doc["movie_list"] = workList

    # print(actor_info_doc)
    # print("-"*30)
    docsList.append(actor_info_doc)
print(docsList)

[{'actor': '안재홍', 'actor_url': 'http://www.cine21.com//db/person/info/?person_id=4883', 'indices': '79451', 'start_period': '2023-04', 'info': {'직업': '배우', '홈페이지': ['https://www.instagram.com/aahnjaehong/'], '생년월일': '1986-03-31', '성별': '남'}, 'movie_list': [['http://www.cine21.com//movie/info/?movie_id=60178', '리바운드', '(2022)'], ['http://www.cine21.com//movie/info/?movie_id=58718', '하이파이브', '(2021)'], ['http://www.cine21.com//movie/info/?movie_id=57060', '울렁울렁 울렁대는 가슴안고', '(2020)']]}, {'actor': '이신영', 'actor_url': 'http://www.cine21.com//db/person/info/?person_id=113283', 'indices': '68100', 'start_period': '2023-04', 'info': {'직업': '배우', '홈페이지': ['https://www.instagram.com/leesin_y/'], '생년월일': '1998-01-24', '성별': '남'}, 'movie_list': [['http://www.cine21.com//db/tv/info/?tv_id=24430', '낭만닥터 김사부 3', '(2023)'], ['http://www.cine21.com//db/tv/info/?tv_id=23303', '너와 나의 경찰수업', '(2022)'], ['http://www.cine21.com//movie/info/?movie_id=60178', '리바운드', '(2022)']]}, {'actor': '박서준', 'actor_url':

In [292]:
docsList

[{'actor': '안재홍',
  'actor_url': 'http://www.cine21.com//db/person/info/?person_id=4883',
  'indices': '79451',
  'start_period': '2023-04',
  'info': {'직업': '배우',
   '홈페이지': ['https://www.instagram.com/aahnjaehong/'],
   '생년월일': '1986-03-31',
   '성별': '남'},
  'movie_list': [['http://www.cine21.com//movie/info/?movie_id=60178',
    '리바운드',
    '(2022)'],
   ['http://www.cine21.com//movie/info/?movie_id=58718', '하이파이브', '(2021)'],
   ['http://www.cine21.com//movie/info/?movie_id=57060',
    '울렁울렁 울렁대는 가슴안고',
    '(2020)']]},
 {'actor': '이신영',
  'actor_url': 'http://www.cine21.com//db/person/info/?person_id=113283',
  'indices': '68100',
  'start_period': '2023-04',
  'info': {'직업': '배우',
   '홈페이지': ['https://www.instagram.com/leesin_y/'],
   '생년월일': '1998-01-24',
   '성별': '남'},
  'movie_list': [['http://www.cine21.com//db/tv/info/?tv_id=24430',
    '낭만닥터 김사부 3',
    '(2023)'],
   ['http://www.cine21.com//db/tv/info/?tv_id=23303', '너와 나의 경찰수업', '(2022)'],
   ['http://www.cine21.com//movi

In [294]:
actors_info.insert_many(docsList)

## mongoDB 조회

In [295]:
for doc in actors_info.find():
    print(doc)

{'_id': ObjectId('645b36a889b7565412185c18'), 'actor': '안재홍', 'actor_url': 'http://www.cine21.com//db/person/info/?person_id=4883', 'indices': '79451', 'start_period': '2023-04', 'info': {'직업': '배우', '홈페이지': ['https://www.instagram.com/aahnjaehong/'], '생년월일': '1986-03-31', '성별': '남'}, 'movie_list': [['http://www.cine21.com//movie/info/?movie_id=60178', '리바운드', '(2022)'], ['http://www.cine21.com//movie/info/?movie_id=58718', '하이파이브', '(2021)'], ['http://www.cine21.com//movie/info/?movie_id=57060', '울렁울렁 울렁대는 가슴안고', '(2020)']]}
{'_id': ObjectId('645b36a889b7565412185c19'), 'actor': '이신영', 'actor_url': 'http://www.cine21.com//db/person/info/?person_id=113283', 'indices': '68100', 'start_period': '2023-04', 'info': {'직업': '배우', '홈페이지': ['https://www.instagram.com/leesin_y/'], '생년월일': '1998-01-24', '성별': '남'}, 'movie_list': [['http://www.cine21.com//db/tv/info/?tv_id=24430', '낭만닥터 김사부 3', '(2023)'], ['http://www.cine21.com//db/tv/info/?tv_id=23303', '너와 나의 경찰수업', '(2022)'], ['http://www.cin

## UPDATE

In [296]:
actors_info.update_many(
    {},
    {"$rename": {"actor": "name"}}
)

In [297]:
for doc in actors_info.find():
    print(doc)

{'_id': ObjectId('645b3f52cc95e8aa99b7fcd5'), 'actor_url': 'http://www.cine21.com//db/person/info/?person_id=4883', 'indices': '79451', 'start_period': '2023-04', 'info': {'직업': '배우', '홈페이지': ['https://www.instagram.com/aahnjaehong/'], '생년월일': '1986-03-31', '성별': '남'}, 'movie_list': [['http://www.cine21.com//movie/info/?movie_id=60178', '리바운드', '(2022)'], ['http://www.cine21.com//movie/info/?movie_id=58718', '하이파이브', '(2021)'], ['http://www.cine21.com//movie/info/?movie_id=54142', '사냥의 시간', '(2020)']], 'name': '안재홍'}
{'_id': ObjectId('645b3f52cc95e8aa99b7fcd6'), 'actor_url': 'http://www.cine21.com//db/person/info/?person_id=113283', 'indices': '68100', 'start_period': '2023-04', 'info': {'직업': '배우', '홈페이지': ['https://www.instagram.com/leesin_y/'], '생년월일': '1998-01-24', '성별': '남'}, 'movie_list': [['http://www.cine21.com//db/tv/info/?tv_id=24430', '낭만닥터 김사부 3', '(2023)'], ['http://www.cine21.com//db/tv/info/?tv_id=23303', '너와 나의 경찰수업', '(2022)'], ['http://www.cine21.com//movie/info/?movi